In [1]:
import pandas as pd
import cv2
import os
import numpy as np

# We have one face !

In [2]:
def FaceOrNot(img) :
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    #print(faces)
    if len(faces) == 1 :
        return True
    else :
        return False

# Vector Extraction

In [3]:
#Model Calling
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer
from model import create_model
import numpy as np
import os.path

Using TensorFlow backend.


In [4]:
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [5]:
#Input Preprocessing
import cv2
from align import AlignDlib

%matplotlib inline

def load_image(path):
    img = cv2.imread(path, 1)
    #print(img)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]

# Initialize the OpenFace face alignment utility
alignment = AlignDlib('landmarks.dat')

def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

# Sample of list of vectors

In [6]:
def ListOfVec(ID) : 
    Resultlist = []
    for i in range(5) :
        path = "images/" + str(ID) + "/opencv_frame_" + str(i+1) + ".jpg"
        #print(path)
        img = load_image(path)
        img = align_image(img)
        #img = (img / 255.).astype(np.float32)
        Resultlist.append(nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0])
        #print(len(Resultlist))
    return Resultlist

# Vector Fetching

In [7]:
#Throw our real product we will be fetching vectors from our csv file but for now 'Testing phase' 
#we will extract our vectors using our functions

In [8]:
VecList1 = np.zeros((100, 128))

In [9]:
#Extraction Phase
def ListOfVec1(ID) : 
    Resultlist = np.zeros((100, 128))
    i = 0
    for w in range(5) :
        path = "images/" + "MyPhotos/" + str(i+1) + ".jpg"
        #print(path)
        img = load_image(path)
        img = align_image(img)
        img = (img / 255.).astype(np.float32)
        Resultlist[w] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
        #print(len(Resultlist))
        i = i + 1
    return Resultlist
VecList1 = VecList1 + ListOfVec1(1)

In [10]:
#Extraction Phase
def ListOfVec2(ID) : 
    Resultlist = np.zeros((100, 128))
    i = 0
    for w in range(5,10) :
        path = "images/" + "AL-Limby/" + str(i+1) + ".jpg"
        #print(path)
        img = load_image(path)
        img = align_image(img)
        #img = (img / 255.).astype(np.float32)
        Resultlist[w] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
        #print(len(Resultlist))
        i = i + 1
    return Resultlist
VecList1 = VecList1 + ListOfVec2(1)

In [11]:
#Extraction Phase
def ListOfVec3(ID) : 
    Resultlist = np.zeros((100, 128))
    i = 0
    for w in range(10,15) :
        path = "images/" + "Colin_Powell/" + str(i+1) + ".jpg"
        #print(path)
        img = load_image(path)
        img = align_image(img)
        #img = (img / 255.).astype(np.float32)
        Resultlist[w] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
        #print(len(Resultlist))
        i = i + 1
    return Resultlist
VecList1 = VecList1 + ListOfVec3(1)

In [12]:
#Extraction Phase
def ListOfVec4(ID) : 
    Resultlist = np.zeros((100, 128))
    i = 0
    for w in range(15,20) :
        path = "images/" + "Ahmed-Helmy/" + str(i+1) + ".jpg"
        #print(path)
        img = load_image(path)
        img = align_image(img)
        #img = (img / 255.).astype(np.float32)
        Resultlist[w] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
        #print(len(Resultlist))
        i = i + 1
    return Resultlist
VecList1 = VecList1 + ListOfVec4(1)

# Classification

In [13]:
TargetNames = ['Amr Tarek','Amr Tarek','Amr Tarek','Amr Tarek','Amr Tarek','lemby','lemby','lemby','lemby','lemby','Colin','Colin','Colin','Colin','Colin','Ahmed Helmy','Ahmed Helmy','Ahmed Helmy','Ahmed Helmy','Ahmed Helmy']

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score

targets = TargetNames
#print(targets)
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)

In [15]:
VecList1 = VecList1[:20]
embedded = VecList1
print(len(embedded))
# 50 train examples of 10 identities (5 examples each)
X_train = embedded[:20]
# 50 test examples of 10 identities (5 examples each)
X_test = embedded[:20]

y_train = y[:20]
y_test = y[:20]

knn = KNeighborsClassifier(n_neighbors=11)
svc = LinearSVC()

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

20


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

# Input Processing

In [16]:
def Veec(img) : 
    img = align_image(img)
    #print(img)
    img = (img / 255).astype(np.float32)
    Resultlist = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    return Resultlist

# Main

In [19]:
def Main():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('FaceID', frame)
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        FaceFlag = FaceOrNot(frame) 
        if FaceFlag == False :
            cv2.putText(frame, "No Faces", (40, 90), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            cv2.imshow('FaceID', frame)
        elif FaceFlag == True : 
            x = knn.predict(Veec(frame).reshape(1, -1))
            ZzZ = encoder.inverse_transform(x)
            print(ZzZ[0])
            cv2.putText(frame, "Face : " + ZzZ[0], (40, 90), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            cv2.imshow('FaceID', frame)
        
    
        if cv2.waitKey(1)%256 == 32: #Space)
            break
    cap.release()
    cv2.destroyAllWindows()

In [20]:
Exit_flag = True
while Exit_flag : 
    try :
        Main()
        Exit_flag = False
    except :
        Exit_flag = True

Amr Tarek
Amr Tarek
Amr Tarek
Amr Tarek
Amr Tarek
Amr Tarek
Amr Tarek
Amr Tarek
